<a href="https://colab.research.google.com/github/emaguiochet/skills-introduction-to-github/blob/main/ema%20session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
# Téléchargement des ressources NLTK nécessaires
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_excel('/content/data_tweets2.xlsx', sheet_name='Sheet 1 - sent_train')
df

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


In [8]:
# Fonction de nettoyage du texte
def clean_text(text):
    # Conversion en minuscules
    text = text.lower()

    # Suppression des URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Suppression des mentions Twitter et hashtags
    text = re.sub(r'@\w+|\$\w+|#\w+', '', text)

    # Suppression des caractères spéciaux et chiffres
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Tokenization (découpe le texte en une liste de mots)
    tokens = word_tokenize(text)

    # Suppression des stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization (Réduit les mots à leur forme de base pour éviter les variations)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

In [9]:
# Application du nettoyage aux tweets
df['cleaned_text'] = df['text'].apply(clean_text)
df

,text,label,cleaned_text
0,$BYND - JPMorgan reels in expectations on Beyo...,0,jpmorgan reel expectation beyond meat
1,$CCL $RCL - Nomura points to bookings weakness...,0,nomura point booking weakness carnival royal c...
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,cemex cut credit suisse jp morgan weak buildin...
3,$ESS: BTIG Research cuts to Neutral https://t....,0,btig research cut neutral
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,funko slide piper jaffray pt cut
...,...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2,week gainer loser stoxx europe dec
9539,Tupperware Brands among consumer gainers; Unil...,2,tupperware brand among consumer gainer unileve...
9540,vTv Therapeutics leads healthcare gainers; Myo...,2,vtv therapeutic lead healthcare gainer myomo b...
9541,"WORK, XPO, PYX and AMKR among after hour movers",2,work xpo pyx amkr among hour mover


In [10]:
df = df.iloc[:, [1, 2]]  # Garde seulement la 2ᵉ et 3ᵉ colonne
df = df.iloc[:, [1, 0]]  # Inverse l'ordre des colonnes restante
df.head(20)

,cleaned_text,label
0,jpmorgan reel expectation beyond meat,0
1,nomura point booking weakness carnival royal c...,0
2,cemex cut credit suisse jp morgan weak buildin...,0
3,btig research cut neutral,0
4,funko slide piper jaffray pt cut,0
5,technipfmc downgraded berenberg called top pic...,0
6,gm loses bull,0
7,deutsche bank cut hold,0
8,cowen cut market perform,0
9,trendforce cut iphone estimate foxconn delay,0


In [11]:
# Conversion en features avec TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['cleaned_text'])

In [12]:
# Préparation des labels pour classification binaire (négatif vs reste)
y_binary = (df['label'] == 0).astype(int)

Pourquoi faire ça ?
✅ 1. Simplifier le problème
Un modèle binaire est plus simple et souvent plus performant qu'un modèle multi-classes.
On peut utiliser des algorithmes standards comme :
Régression logistique binaire
SVM binaire
Random Forest binaire

✅ 2. Cas d'usage courant : Détection des tweets négatifs
Si on veut prédire si un tweet est négatif ou non, cette approche est parfaite.
Par exemple, une entreprise peut vouloir :
Identifier les clients mécontents sur Twitter.
Détecter les commentaires négatifs pour améliorer son service client.

✅ 3. Éviter le déséquilibre des classes
Si la classe "neutre" ou "positive" est trop surdimensionnée, un modèle multi-class risque de mal apprendre.
En fusionnant neutre et positif en une seule classe (0), on équilibre mieux les données.

In [14]:
# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

print("Dimensions des données d'entraînement:", X_train.shape)
print("Dimensions des données de test:", X_test.shape)
print(X_train[:5].toarray())  # Convertir la matrice en tableau pour affichage
print(np.unique(y_train, return_counts=True))  # Voir la répartition des classes


Dimensions des données d'entraînement: (7634, 10000)
Dimensions des données de test: (1909, 10000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(array([0, 1]), array([6492, 1142]))


* X → Les caractéristiques (features) de tes données (par exemple, du texte transformé en vecteurs TF-IDF).
* y_binary → Les étiquettes (labels) associées (par exemple, 0 = négatif, 1 = positif).
* X_train et X_test → Ce sont des sous-ensembles de X après division.
* y_train et y_test → Ce sont les étiquettes correspondantes à X_train et X_test.

* 7634 documents pour l'entraînement
* 1909 pour le test
* chaque document est représenté par 10000 caractéristiques TF-IDF

---
Pourquoi faire un split ?

 Éviter le surapprentissage : Si on entraîne et teste sur les mêmes données, le modèle pourrait juste "mémoriser" les exemples.

  Évaluer la performance : L'ensemble de test permet de mesurer si le modèle généralise bien sur de nouvelles données.


diviser les données en un ensemble d'entraînement (80%) et un ensemble de test(20%) pour entraîner et évaluer un modèle de machine learning

**random_state=42** assure que la division est répétable (pour obtenir les mêmes résultats à chaque exécution).

* **X** : Les features (ici, la matrice TF-IDF du texte).
* **y_binary**: Les étiquettes (ex. : 0 ou 1 si c'est un problème de classification binaire).
* **X_train**: 80% des données pour entraîner le modèle.
* **X_test** : 20% des données pour tester le modèle.
* **y_train**: Les étiquettes associées à X_train.
* **y_test** : Les étiquettes associées à X_test.





In [15]:
# Affichage des premiers exemples nettoyés
print("\nExemples de tweets nettoyés:")
print(df['cleaned_text'].head())


Exemples de tweets nettoyés:
0                jpmorgan reel expectation beyond meat
1    nomura point booking weakness carnival royal c...
2    cemex cut credit suisse jp morgan weak buildin...
3                            btig research cut neutral
4                     funko slide piper jaffray pt cut
Name: cleaned_text, dtype: object


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [17]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Mesure du temps d'exécution
    start_time = time.time()

    # Entraînement du modèle
    model.fit(X_train, y_train)

    # Prédictions
    y_pred = model.predict(X_test)

    # Temps d'exécution
    execution_time = time.time() - start_time

    # Métriques
    print(f"\nRésultats pour {model_name}")
    print("Temps d'exécution: {:.2f} secondes".format(execution_time))
    print("\nRapport de classification:")
    print(classification_report(y_test, y_pred))

# Définition des modèles
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500),
    'SVM': SVC(),  # Modèle supplémentaire 1
    'Naive Bayes': MultinomialNB()  # Modèle supplémentaire 2
}

# Évaluation de chaque modèle
for model_name, model in models.items():
    evaluate_model(model, X_train, X_test, y_train, y_test, model_name)


Résultats pour Logistic Regression
Temps d'exécution: 0.06 secondes

Rapport de classification:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1609
           1       0.92      0.20      0.32       300

    accuracy                           0.87      1909
   macro avg       0.90      0.60      0.63      1909
weighted avg       0.88      0.87      0.83      1909


Résultats pour KNN
Temps d'exécution: 0.35 secondes

Rapport de classification:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1609
           1       0.90      0.06      0.12       300

    accuracy                           0.85      1909
   macro avg       0.88      0.53      0.52      1909
weighted avg       0.86      0.85      0.79      1909


Résultats pour Random Forest
Temps d'exécution: 22.21 secondes

Rapport de classification:
              precision    recall  f1-score   support

           0       0.

* Precision:	% de prédictions correctes parmi celles qui ont été classées dans cette catégorie.(Quand le modèle dit "0", il a raison 87% du temps)
* Recall:	% d'éléments d'une classe correctement identifiés.(0.20 → Il n'a trouvé que 20% des vrais "0")
* F1-score:	Moyenne harmonique entre précision et rappel.
* Support:	Nombre total d'exemples dans cette classe.
__________________
* Accuracy = 0.87 → Le modèle classe correctement 87% des échantillons
* Macro avg (moyenne des classes) → Score équilibré mais faible pour le rappel (0.60).
* Weighted avg (pondéré par le support) → Donne plus d'importance à la classe majoritaire (0), d'où un bon f1-score global.


1️⃣ Logistic Regression

Temps d'exécution: ⏱️ 0.14 secondes Résultats:

Precision (classe 1): 92%
Recall (classe 1): 19% (très faible)
F1-score (classe 1): 32%
Accuracy globale: 87%

Interprétation:

La régression logistique classifie bien la classe majoritaire (0) avec un recall de 100%.
Par contre, elle ne détecte presque pas la classe 1 (recall = 19%). Cela signifie que beaucoup d'exemples de classe 1 sont classés à tort comme 0.
Ce modèle est biaisé en faveur de la classe majoritaire.

2️⃣ K-Nearest Neighbors (KNN)

Temps d'exécution: ⏱️ 0.30 secondes Résultats:

Precision (classe 1): 86%
Recall (classe 1): 6% (très faible)
F1-score (classe 1): 11%
Accuracy globale: 85%

Interprétation:

KNN a du mal avec la classe minoritaire (1) : il ne détecte presque aucun exemple de cette classe (recall de seulement 6%).
Cela signifie que presque tous les exemples de classe 1 sont mal classés comme 0.
KNN a tendance à sur-apprendre les classes majoritaires, surtout si les données sont déséquilibrées.

3️⃣ Random Forest

Temps d'exécution: ⏱️ 11.80 secondes Résultats:

Precision (classe 1): 83%
Recall (classe 1): 32%
F1-score (classe 1): 46%
Accuracy globale: 88%

Interprétation:

Random Forest fait mieux que KNN et Logistic Regression sur la classe 1 (recall de 32%, contre 6% pour KNN et 19% pour Logistic Regression).
Son temps d'exécution est plus long car il entraîne 100 arbres de décision.
Bilan : Un bon équilibre, mais il reste biaisé vers la classe majoritaire.

4️⃣ Neural Network (MLPClassifier)

Temps d'exécution: ⏱️ 53.99 secondes Résultats:

Precision (classe 1): 67%
Recall (classe 1): 48%
F1-score (classe 1): 56%
Accuracy globale: 88%

Interprétation:

Ce modèle fait mieux que tous les autres sur la classe minoritaire (1) avec un recall de 48%.
Son temps d'exécution est long car un réseau de neurones fait plusieurs passes sur les données (backpropagation).
Il a une meilleure capacité de généralisation mais peut nécessiter encore plus d'optimisation (nombre de couches, nombre d'itérations, régularisation).

5️⃣ Support Vector Machine (SVM)

Temps d'exécution: ⏱️ 3.01 secondes Résultats:

Precision (classe 1): 95%
Recall (classe 1): 26%
F1-score (classe 1): 40%
Accuracy globale: 88%

Interprétation:

SVM a une très bonne précision sur la classe minoritaire (95%), mais son recall est faible (26%).
Cela signifie qu'il classe bien les exemples de classe 1 quand il les détecte, mais il en rate beaucoup.
Son temps d'exécution est raisonnable par rapport à Random Forest et Neural Network.

6️⃣ Naive Bayes

Temps d'exécution: ⏱️ 0.00 secondes (extrêmement rapide) Résultats:

Precision (classe 1): 100%
Recall (classe 1): 2% (très mauvais)
F1-score (classe 1): 4%
Accuracy globale: 85%

Interprétation:

Ce modèle prédit la classe 0 presque tout le temps (recall de 2% pour la classe 1).
Il n'est pas adapté aux données déséquilibrées et suppose une indépendance forte entre les mots (ce qui n'est pas réaliste pour du texte).
Son avantage est qu'il est très rapide (0.00 sec).

    Si tu veux de la rapidité : Logistic Regression ou Naive Bayes
    Si tu veux détecter la classe 1 plus efficacement : Neural Network
    Si tu veux un bon compromis : Random Forest ou SVM

Le modèle Neural Network semble être le meilleur pour équilibrer précision et recall, mais il est plus long à entraîner.


#2 grading criterion 2: Model Optimization
##2.1 Selection of Explanatory Variables

#Neural Network model


In [43]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3) )  # features: caractéristiques #ngram: taille des groupes de mots (de 1 à 3mots)
X = vectorizer.fit_transform(df['cleaned_text'])
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

💡 Problème principal : déséquilibre de classes

La classe 1 est moins représentée (seulement 300 exemples contre 1609 pour la classe 0) → le modèle penche fort vers la classe 0.
Nous allons donc rééquilibrer:

##Sur-échantillonage  de la classe minoritaire
avec SMOTE

In [51]:
!pip install -q imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# SMOTE sur l'ensemble d'entraînement
smote = SMOTE(random_state=42) #regarde les instances de la classe minoritaire dans y_train
#Il génère artificiellement de nouveaux exemples synthétiques de cette classe en interpolant entre les exemples existants.
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
#les données sont rééchantillonnées : la classe minoritaire est maintenant aussi représentée que la classe majoritaire.

nn_model = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=500, random_state=42)
evaluate_model(nn_model, X_train_res, X_test, y_train_res, y_test, "Neural Network (SMOTE)")


Résultats pour Neural Network (SMOTE)
Temps d'exécution: 281.27 secondes

Rapport de classification:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1609
           1       0.64      0.54      0.58       300

    accuracy                           0.88      1909
   macro avg       0.78      0.74      0.76      1909
weighted avg       0.87      0.88      0.88      1909



Il n'y a pas un grand chagement comparé au modèle précédent, et il prend beaucoup plus de temps

#Sous-échantillonage de la classe majoritaire
avec la RandomUnderSampler

In [46]:
from imblearn.under_sampling import RandomUnderSampler

#équilibre le jeu de données déséquilibré en réduisant la taille de la classe majoritaire (supprime de manière aléatoire des données de la classe majoritaire).
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

nn_model = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=500, random_state=42)
evaluate_model(nn_model, X_train_res, X_test, y_train_res, y_test, "Neural Network (Undersampled)")

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7edd4ded0040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e


Résultats pour Neural Network (Undersampled)
Temps d'exécution: 42.71 secondes

Rapport de classification:
              precision    recall  f1-score   support

           0       0.94      0.78      0.85      1609
           1       0.39      0.74      0.51       300

    accuracy                           0.77      1909
   macro avg       0.66      0.76      0.68      1909
weighted avg       0.85      0.77      0.80      1909



Le nouveau modèle est **meilleur si notre objectif est de ne pas rater les exemples de la classe 1 (fort recall).
Mais il est moins bon globalement et il prédit trop souvent à tort la classe 1 (faible précision).

✅ Le recall de la classe 1 s’est nettement amélioré : de 0.53 à 0.74. Donc le modèle avec SMOTE détecte beaucoup plus de vrais positifs pour la classe minoritaire.

❌ Mais la précision sur la classe 1 a chuté : de 0.67 à 0.39 → beaucoup plus de faux positifs.

❌ Le f1-score de la classe 1 a baissé (de 0.59 à 0.51) car la précision a trop chuté.

❌ L’accuracy globale a baissé (de 0.89 à 0.77), car le modèle fait maintenant plus d’erreurs sur la classe majoritaire (0).